## __SOLUCION AWS CON BUCKET__

1) subi documentacion s3buket

2) texttrac conectar

3) eliminar documento del bucket

__CREAR DOCUMENTO S3BUCKET__

In [1]:
import boto3
s3 = boto3.resource('s3')

# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

elasticbeanstalk-us-east-2-911499329769
textract-console-us-east-2-a297ace1-84df-4e82-9c0e-20034e07410b


In [6]:
import glob

docs_pdf =  glob.glob('../Info para entregar/*.pdf')  #glob.glob("i/*.pdf")

In [7]:
len(docs_pdf)

50

__SUBIR DOCUMENTOS__

In [8]:
import PyPDF2 

s3BucketName = 'textract-console-us-east-2-a297ace1-84df-4e82-9c0e-20034e07410b'

def PUT_BUCKET_DOCUMENTS(pdf_list):
    for pdf in range(len(pdf_list)):
        pdfFileObj = open(pdf_list[pdf], 'rb') 
        #subir bucket
        s3.Bucket(s3BucketName).put_object(Key= 'ejemplo_' +str(pdf) + '.pdf', Body = pdfFileObj )
    return 'all ready 1)'


In [9]:
PUT_BUCKET_DOCUMENTS(docs_pdf[4:8])

'all ready 1)'

__TEXTRACT AWS__

In [10]:
import boto3 

# Document
s3BucketName = 'textract-console-us-east-2-a297ace1-84df-4e82-9c0e-20034e07410b'
documentName = 'ejemplo_0.pdf'

# Amazon Textract client
textract = boto3.client('textract')

# Call Amazon Textract
response = textract.analyze_document(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    },
    FeatureTypes = ["TABLES","FORMS"])


InvalidS3ObjectException: An error occurred (InvalidS3ObjectException) when calling the AnalyzeDocument operation: Unable to get object metadata from S3. Check object key, region and/or access permissions.

__EXTRER EN TEXTO__

In [ ]:
#optimiza la lectura, detecta cuales son LINE
#indice_line = []
#for ii in range(len(response["Blocks"])):
#    if response["Blocks"][ii]["BlockType"] =="LINE":
#        indice_line.append(ii)
        
#lee segun las lineas

def PDF_Extraer_Texto(PDF_RESPONSE):
    ## recorre pdf ###
    PDF_texto = []
    for pdf in range(len(PDF_RESPONSE)):
        PAGE_texto = []
        for page  in range(len(PDF_RESPONSE[pdf])):
            ## recorre paginas ##
            response = PDF_RESPONSE[pdf][page]
            text_ini = ''
            for ii in range(len(response["Blocks"])):
                if response["Blocks"][ii]["BlockType"] =="LINE":
                    text_ini = text_ini + '\n' +  response["Blocks"][ii]["Text"] + '\n'  
            PAGE_texto.append(text_ini)
        PDF_texto.append(PAGE_texto)
    return PDF_texto
    

In [ ]:
%%time
TEXTO_PDFS = PDF_Extraer_Texto(final_responses)

__EXTRAER TABLA__

In [ ]:
from trp import Document
import pandas as pd
import numpy as np


def Retorna_tabla(response):
    
    doc = Document(response)
    #definicio de filas
    rows = len(doc.pages[0].tables[0].rows)

    #definicion de columnas
    if len(doc.pages[0].tables[0].rows[0].cells) == len(doc.pages[0].tables[0].rows[rows-1].cells):
        cols = len(doc.pages[0].tables[0].rows[rows-1].cells)
    else:
        cols = max(len(doc.pages[0].tables[0].rows[0].cells),len(doc.pages[0].tables[0].rows[rows-1].cells))

    tabla_doc = pd.DataFrame(index=np.arange(cols), columns=np.arange(rows))

    for page in doc.pages:
         # Print tables
        for table in page.tables:
            for r, row in enumerate(table.rows):
                for c, cell in enumerate(row.cells):
                    tabla_doc[r][c] = doc.pages[0].tables[0].rows[r].cells[c].text
                
    return tabla_doc.transpose()

In [ ]:
def TABLAS_pdf(PDF_RESPONSE):
    ## recorre pdf ###
    PDF_tablas = []
    for pdf in range(len(PDF_RESPONSE)):
        print(pdf)
        PAGE_tabla = []
        for page  in range(len(PDF_RESPONSE[pdf])):
            ## recorre paginas ##
            response_page = PDF_RESPONSE[pdf][page]
            text_ini = Retorna_tabla(response=response_page)
            PAGE_tabla.append(text_ini)
        PDF_tablas.append(PAGE_tabla)
    return PDF_tablas


In [ ]:
len(final_responses)

In [ ]:
%%time

PDF_TABLAS = TABLAS_pdf(final_responses)

__ELIMINAR DE LA NUBE__

In [10]:

def DEL_BUCKET_DOCUMENTS(pdf_list):
    for pdf in range(len(pdf_list)):
        s3.Object(s3BucketName, 'ejemplo_' +str(pdf) + '.pdf' ).delete()
    return 'all ready 4)'


In [11]:
DEL_BUCKET_DOCUMENTS(docs_pdf[0:3])

'all ready 4)'